In [32]:
import os
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from function import *
from function_preparation import *

1. Data Preparation

#1.1 Xử lý df

In [33]:
eth_path = r"E:\MyPythonCode\Crypto\data\csv\binance_ETHUSDT.csv"
df = pd.read_csv(eth_path)
df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
df = df.resample('1h', on='Open Time').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Quote Volume': 'sum',
    'Number of Trades': 'sum',
    'Taker Buy Base Volume': 'sum',
    'Taker Buy Quote Volume': 'sum',
}).dropna()
df['SMA_10'] = calculate_sma(df, period=10)
df['EMA_10'] = calculate_ema(df, period=10)
df['RSI_14'] = calculate_rsi(df, period=14)
df['MACD'], df['Signal_Line'] = calculate_macd(df)
df['Bollinger_Upper'], df['Bollinger_Lower'] = calculate_bollinger_bands(df, period=20)
df['ATR'] = calculate_atr(df, period=14)
df['Stochastic_Oscillator'] = calculate_stochastic_oscillator(df, period=14)
df['OBV'] = calculate_obv(df)
df['ROC'] = calculate_roc(df, period=12)
df['CMF'] = calculate_cmf(df, period=20)
df['ADX_14'] = calculate_adx(df, period=14)
df['SAR'] = calculate_sar(df, acceleration=0.02, maximum=0.2)
df = df.dropna()
df = df.iloc[20:].reset_index(drop=True)
print(df)
# df.to_excel('file_name.xlsx', index=False)

         Open     High      Low    Close      Volume  Quote Volume  \
0     1576.70  1577.00  1573.67  1576.03   5061.7066  7.973899e+06   
1     1576.03  1589.74  1576.03  1584.64  15733.8816  2.493128e+07   
2     1584.64  1597.50  1584.63  1590.61  17918.3936  2.853673e+07   
3     1590.61  1595.08  1588.08  1592.46   6730.2225  1.070817e+07   
4     1592.46  1594.00  1590.85  1591.28   3590.0051  5.716626e+06   
...       ...      ...      ...      ...         ...           ...   
8709  2580.15  2584.77  2572.05  2580.59   7885.5562  2.035016e+07   
8710  2580.59  2585.53  2575.12  2581.17   4044.7767  1.043823e+07   
8711  2581.17  2588.00  2566.19  2570.70  12832.4517  3.304580e+07   
8712  2570.70  2574.60  2554.82  2564.04  13095.4564  3.358454e+07   
8713  2564.04  2576.20  2562.24  2571.82   4115.0255  1.057446e+07   

      Number of Trades  Taker Buy Base Volume  Taker Buy Quote Volume  \
0                15379              2692.3562            4.241202e+06   
1            

#1.2 Tạo đầu vào cho mô hình

In [34]:
timesteps = 24*7 
X = []
y = []

In [35]:
for i in range(len(df) - timesteps):
    # Giống như việc quét chọn 1 khu vực trong bảng tính, chỉ định index từ đâu đến đâu    

    # Lấy các cột đặc trưng làm X
    X.append(df.iloc[i:i+timesteps].values)  
    # Tất cả cột là đầu vào
    # *** Giải thích ***
    # [tức là lấy hàng từ (i) đến (i+timesteps-1), và cột giá trị là cả df
        
    # Lấy cột 'Label' ở vị trí i+timesteps làm y
    y.append(df.iloc[i+timesteps]['Close'])  
    # *** Giải thích ***
    # [tức là lấy hàng (i+timesteps), và cột giá trị là 'Close']

X, y = np.array(X), np.array(y)
print_comboXy_shape(X, y)

# Chia dữ liệu thành train (70%), validation (15%), và test (15%)
train_size = int(0.8 * len(X))
val_size = int(0.15 * len(X))
test_size = len(X) - train_size - val_size

# Chia ra train, validation, và test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(val_size + test_size), shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size, shuffle=False)
print(f'Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}')

# Reshape X_train, X_val, X_test từ 3D sang 2D
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test= X_test.reshape(X_test.shape[0], -1)

# Kiểm tra lại shape
print("Sau khi reshape cho phù hợp mới MachineLearning")
print(f'Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}')


X shape: (8546, 168, 34)
y shape: (8546,)
Train shape: (6836, 168, 34), Val shape: (1281, 168, 34), Test shape: (429, 168, 34)
Sau khi reshape cho phù hợp mới MachineLearning
Train shape: (6836, 5712), Val shape: (1281, 5712), Test shape: (429, 5712)


2. Dùng Random-Forest thủ công

In [36]:
from classML import *
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
from sklearn.metrics import mean_absolute_error

2.1 Khởi tạo RF với khả năng tái huấn luyện (warm_start=True)

In [37]:
RF = RandomForestRegressor(n_estimators=100, max_depth=6, warm_start=True)

2.2 Train như bình thường

In [38]:
RF.fit(X_train, y_train)
savepath = 'modelsML\\random_forest_with_warm_start.pkl'
joblib.dump(RF, savepath)

['modelsML\\random_forest_with_warm_start.pkl']

2.3 Dự đoán và đánh giá độ chính xác (Step: evaluate)

In [40]:
#2.3.1 Tải lại mô hình từ tệp
RF_loaded = joblib.load(savepath)

In [41]:
#2.3.2 Chọn bộ dữ liệu để test
mode = 'test'
if mode == 'test':
    checkX = X_test
    checky = y_test
else:
    checkX = X_val
    checky = y_val

In [42]:
#2.3.3 Thay thế cặp X_test, y_test thành X_val, y_val
y_pred = RF_loaded.predict(checkX)        
                
# Kiểm tra hiệu suất của mô hình
mse = mean_squared_error(checky, y_pred)
rmse = mean_squared_error(checky, y_pred, squared=False)
mae = mean_absolute_error(checky, y_pred)
print("Trung bình bình phương của sai số: ", mse)
print("Sai số thực tế (with outliers): ", rmse)
print("Sai số trung bình (without outliers): ", mae)

Trung bình bình phương của sai số:  242.61472883056481
Sai số thực tế (with outliers):  15.576094787544303
Sai số trung bình (without outliers):  11.91908751452804


e:\MyPythonCode\Crypto\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


2.4 Retrain nếu chưa đủ tốt

# Lưu ý:
chỉ thêm các cây khác vào models chứ không thay đổi các cây cũ

In [39]:
#2.4.1 Tải lại mô hình từ tệp
RF_loaded = joblib.load(savepath)

In [ ]:
# Tăng số lượng cây trong rừng
RF_loaded.n_estimators += 50  # Thêm 50 cây nữa
RF_loaded.fit(X_train, y_train)  # Tiếp tục huấn luyện

2.5 Lặp lại bước 2.3, nhưng với model RF mới cải tiến